In [6]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    pipeline
)

In [13]:
model_id="FacebookAI/roberta-base"
# relace the value with your model: ex <hugging-face-user>/<model-name>

In [12]:
classifier= pipeline('text-classification', model=model_id)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
classifier("HELLO!!")

[{'label': 'LABEL_1', 'score': 0.5726537704467773}]

In [87]:
import pandas as pd
df1 = pd.read_csv("./jira_scrum_role_dataset.csv")
df2 = pd.read_csv("./role_classification_dataset_long.csv")

In [89]:
dataset = pd.concat([df1,df2], ignore_index=True)

In [90]:
dataset

,description,role
0,Implement responsive UI using React: Implement...,FrontEndEngineer
1,Add new components to the dashboard: Add new c...,FrontEndEngineer
2,Add new components to the dashboard: Add new c...,FrontEndEngineer
3,Fix CSS styling issues for mobile view: Fix CS...,FrontEndEngineer
4,Add new components to the dashboard: Add new c...,FrontEndEngineer
...,...,...
6995,Scale bleeding-edge web-readiness. Develop new...,FrontEndEngineer
6996,Revolutionize scalable solutions. Configure al...,DevOpsEngineer
6997,Syndicate customized paradigms. Improve access...,FrontEndEngineer
6998,Incubate mission-critical architectures. Fix b...,BackendEngineer


In [96]:
dataset_y = dataset['role']
dataset.drop(['role'], inplace=True, axis = 1)

In [98]:
dataset_y

0       FrontEndEngineer
1       FrontEndEngineer
2       FrontEndEngineer
3       FrontEndEngineer
4       FrontEndEngineer
              ...       
6995    FrontEndEngineer
6996      DevOpsEngineer
6997    FrontEndEngineer
6998     BackendEngineer
6999     BackendEngineer
Name: role, Length: 7000, dtype: object

In [93]:
from sklearn.model_selection import train_test_split

In [99]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_y, test_size=0.33, random_state=42)

In [121]:
X_train, y_train
# y_train.unique()

array(['CloudEngineer', 'DevOpsEngineer', 'AIEngineer',
       'DatabaseDesignEngineer', 'BackendEngineer', 'FrontEndEngineer'],
      dtype=object)

## Preprocessing

In [122]:
cols = ['CloudEngineer', 'DevOpsEngineer', 'AIEngineer',
       'DatabaseDesignEngineer', 'BackendEngineer', 'FrontEndEngineer']

In [124]:
from transformers import RobertaTokenizerFast
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

# 1. Convert DataFrame columns to lists
texts = X_train.iloc[:, 0].tolist()  # or X_train['description'].tolist() if named
labels = cols  # or Y_train['role'].tolist()

# 2. Encode labels (e.g., 'AIEngineer' → 0, etc.)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 3. Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# 4. Tokenize texts
encodings = tokenizer(
    texts,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

# 5. Create custom Dataset class
class RoleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# 6. Prepare the dataset
train_dataset = RoleDataset(encodings, cols)


ValueError: too many dimensions 'str'

6842    [input_ids, attention_mask]
2103    [input_ids, attention_mask]
5500    [input_ids, attention_mask]
2646    [input_ids, attention_mask]
3739    [input_ids, attention_mask]
                   ...             
3772    [input_ids, attention_mask]
5191    [input_ids, attention_mask]
5226    [input_ids, attention_mask]
5390    [input_ids, attention_mask]
860     [input_ids, attention_mask]
Name: description, Length: 4690, dtype: object